In [70]:
import my_utils2 as mu

In [40]:
url = 'http://openapi.seoul.go.kr:8088/(인증키)/xml/tbLnOpendataRentV/1/5/'

In [41]:
df = mu.get_df(url)

In [6]:
df

,ACC_YEAR,SGG_CD,SGG_NM,BJDONG_CD,BJDONG_NM,LAND_GBN,LAND_GBN_NM,BOBN,BUBN,FLR_NO,...,RENT_GTN,RENT_FEE,BLDG_NM,BUILD_YEAR,HOUSE_GBN_NM,CNTRCT_PRD,NEW_RON_SECD,CNTRCT_UPDT_RQEST_AT,BEFORE_GRNTY_AMOUNT,BEFORE_MT_RENT_CHRGE
0,2023,11290,성북구,13700,상월곡동,,,,,0.0,...,15000,0,,1994,단독다가구,24.01~26.01,신규,,0,
1,2023,11740,강동구,10800,성내동,,,,,0.0,...,8000,44,,2016,단독다가구,23.12~25.12,갱신,○,8000,35
2,2023,11230,동대문구,10600,장안동,,,,,0.0,...,15000,0,,1993,단독다가구,,,,,
3,2023,11350,노원구,10500,상계동,,,,,0.0,...,1000,35,,1995,단독다가구,24.01~26.01,신규,,0,0
4,2023,11380,은평구,10700,응암동,,,,,0.0,...,14700,0,,1996,단독다가구,23.12~25.12,갱신,,14000,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023,11710,송파구,10900,장지동,1,대지,0905,0000,14.0,...,6669,11,위례포레샤인23단지,2017,아파트,,신규,,0,0
996,2023,11740,강동구,10800,성내동,,,,,0.0,...,23000,0,,1993,단독다가구,,,,,
997,2023,11350,노원구,10300,공릉동,1,대지,0081,0000,5.0,...,23600,0,태강아파트(아이파크),1999,아파트,24.03~26.03,신규,,0,
998,2023,11215,광진구,10700,화양동,,,,,0.0,...,20500,0,,1997,단독다가구,23.12~25.12,갱신,,19000,


In [42]:
sel_c = ['SGG_NM','BJDONG_NM','LAND_GBN_NM','FLR_NO','CNTRCT_DE','RENT_GBN','RENT_AREA','RENT_GTN','RENT_FEE','BLDG_NM','BUILD_YEAR']

In [43]:
my_df = df[sel_c]

In [44]:
my_df.columns=['자치구명','법정동명','지번구분명','층수','계약일','전월세','면적','보증금','임대료','건물명','건축년도']

In [45]:
my_df = my_df.astype({'층수':'float','계약일':'int','면적':'float','보증금':'int','임대료':'int'})

In [90]:
query ="""
CREATE TABLE HOUSEPRICE(
자치구명 VARCHAR(50),
법정동명 VARCHAR(50),
지번구분명 VARCHAR(50),
층수 FLOAT,
계약일 INT,
전월세 VARCHAR(20),
면적 FLOAT,
보증금 INT,
임대료 INT,
건물명 VARCHAR(100),
건축년도 VARCHAR(20)
)
"""

In [88]:
conn = mu.connect_mysql()

In [91]:
mu.sql_execute(conn,query)
conn.commit()
conn.close()

()

In [92]:
conn.commit()

In [93]:
conn.close()

In [94]:
query2 = (
    f'insert into housePrice(자치구명, 법정동명, 지번구분명, 층수, 계약일, 전월세, 면적, 보증금, 임대료, 건물명, 건축년도)'
    f'values ({repr(my_df.자치구명.iloc[0])},{repr(my_df.법정동명.iloc[0])},{repr(my_df.지번구분명.iloc[0])},{my_df.층수.iloc[0]},'
    f'{my_df.계약일.iloc[0]},{repr(my_df.전월세.iloc[0])},{my_df.면적.iloc[0]},{my_df.보증금.iloc[0]},{my_df.임대료.iloc[0]},'
    f'{repr(my_df.건물명.iloc[0])},{repr(my_df.건축년도.iloc[0])})'
)

In [21]:
repr(my_df.자치구명.iloc[0])

"'성북구'"

In [157]:
conn = mu.connect_mysql()

In [96]:
mu.sql_execute(conn,query2)

()

In [97]:
conn.commit()

In [36]:
my_df.iloc[999]

자치구명             강서구
법정동명             화곡동
지번구분명             대지
층수              15.0
계약일         20231212
전월세               월세
면적             17.04
보증금             1000
임대료               73
건물명      원타워 5차 오피스텔
건축년도            2021
Name: 999, dtype: object

# tqdm

In [80]:
from tqdm import tqdm
tqdm.pandas()

In [67]:
for i in tqdm(range(10000000)):
    pass

100%|██████████| 10000000/10000000 [00:03<00:00, 3013200.73it/s]


In [82]:
def insert(x,conn):
    query2 = (
        f'insert into housePrice(자치구명, 법정동명, 지번구분명, 층수, 계약일, 전월세, 면적, 보증금, 임대료, 건물명, 건축년도)'
        f'values ({repr(x.자치구명)},{repr(x.법정동명)},{repr(x.지번구분명)},{x.층수},'
        f'{x.계약일},{repr(x.전월세)},{x.면적},{x.보증금},{x.임대료},'
        f'{repr(x.건물명)},{repr(x.건축년도)})'
    )
    mu.sql_execute(conn,query2)
    return

In [99]:
conn = mu.connect_mysql()
my_df.progress_apply(lambda x:insert(x, conn), axis = 1)
conn.commit()
conn.close()

100%|██████████| 1000/1000 [00:01<00:00, 642.62it/s]


In [101]:
from sklearn.datasets import load_wine
import pandas as pd

In [100]:
a = load_wine()

In [105]:
df = pd.DataFrame(a['data'], columns = a['feature_names'])

In [110]:
df['drunk'] = 'none'

In [ ]:
df['drunk'] = df.apply(lambda x : 'high' if x['alcohol'] >= 12 else 'lower', axis = 1)

# sklearn에서 가져온 와인데이터를 사용

# drunk 컬럼 생성

# alcohol 컬럼의 값이 12보다 낮을경우 drunk컬럼에 low, 이상이면 high

# mysql에서 테이블생성 후 insert하는 구문을 작성한 뒤, 실제로 insert까지 하기

In [123]:
def per(x):
    if x.alcohol < 12:
        return 'low'
    else:
        return 'high'

In [124]:
df['drunk'] = df.apply(lambda x: per(x), axis = 1)

In [187]:
df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280_od315_of_diluted_wines,proline,drunk
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,high
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,high
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,high
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,high
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,high
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,high
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,high
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,high


In [188]:
query ="""
CREATE TABLE WINE(
alcohol INT,
malic_acid INT,0
ash INT,
alcalinity_of_ash INT,
magnesium INT,
total_phenols INT,
flavanoids INT,
nonflavanoid_phenols INT,
proanthocyanins INT,
color_intensity INT,
hue INT,
od280_od315_of_diluted_wines INT,
proline int,
drunk varchar(10)
)
"""

In [189]:
mu.sql_execute(conn,query)
conn.commit()
conn.close()

In [185]:
def insert(x,conn):
    query2 = (
        f'insert into wine(alcohol, malic_acid, ash, alcalinity_of_ash, magnesium, total_phenols, flavanoids, nonflavanoid_phenols, proanthocyanins, color_intensity, hue, od280_od315_of_diluted_wines, proline, drunk)'
        f'values ({(x.alcohol)},{(x.malic_acid)},{(x.ash)},{x.alcalinity_of_ash},'
        f'{x.magnesium},{(x.total_phenols)},{x.flavanoids},{x.nonflavanoid_phenols},{x.proanthocyanins},'
        f'{(x.color_intensity)},{(x.hue)},{(x.od280_od315_of_diluted_wines)}, {(x.proline)}, {repr(x.drunk)})'
    )
    mu.sql_execute(conn,query2)
    return

In [181]:
df = df.rename(columns={'od280/od315_of_diluted_wines': 'od280_od315_of_diluted_wines'})

In [190]:
conn = mu.connect_mysql()
df.progress_apply(lambda x:insert(x, conn), axis = 1)
conn.commit()
conn.close()

100%|██████████| 178/178 [00:00<00:00, 350.58it/s]


TypeError: Index does not support mutable operations

In [178]:
df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,drunk
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,high
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,high
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,high
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,high
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,high
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,high
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,high
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,high
